In [1]:
import pandas as pd
import re
from collections import Counter
import jieba

filename = 'movie_comments.csv'
content = pd.read_csv(filename)
articles = content['comment'].tolist()

def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)

articles_clean = [''.join(token(str(a)))for a in articles]
                      
with open('article_9k.txt', 'w',encoding='utf-8') as f:
    for a in articles_clean:
        f.write(a + '\n')
                      
def cut(string): 
    return list(jieba.cut(string))
                      
TOKEN = []
for i, line in enumerate((open('article_9k.txt',encoding='utf-8'))):
    if i % 10000 == 0: print(i)
    # replace 10000 with a big number when you do your homework. 
    TOKEN += cut(line)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Windows\Temp\jieba.cache


0


Loading model cost 1.225 seconds.
Prefix dict has been built succesfully.


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000


In [2]:

words_count = Counter(TOKEN)

TOKEN = [str(t) for t in TOKEN]
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]
words_count_2 = Counter(TOKEN_2_GRAM)

def prob_1(word): 
    return words_count[word] / len(TOKEN)
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: return words_count_2[word1+word2] / words_count[word2]
    else:
        return 1 / len(TOKEN_2_GRAM)
def get_probability(sentence):
    words = cut(sentence)
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        probability = prob_2(word, next_)
        sentence_pro *= probability
    sentence_pro *= prob_1(words[-1])
    return sentence_pro

In [3]:
eat = """
aim => time who action food_1 cooking food_2 ， how
time => 今天 | 昨天 | 前天
who => 我 | 他 | 小明 | 红红 | 龟龟 | 方方
action => 吃了 | 尝了
food_1 => 西红柿 | 尖椒 | 木耳 | 腐竹 | 青椒 | 黄豆拌饭酱
cooking => 炒 | 凉拌
food_2 => 鸡蛋 | 皮蛋 | 肉丝 | 土豆丝 | 茄子 | 油菜
how => 好吃 | 不好吃 | 你猜好不好吃 | 你尝尝
"""

import random

def generate_grammar(grammar_str: str, target, split='=>'):
    grammar = {}   
    for line in grammar_str.split('\n'):
        if not line: continue
        expression, formula = line.split(split)
        formulas = formula.split('|')
        formulas = [f.split() for f in formulas]
        grammar[expression.strip()] = formulas        
#        print(expression, ':', formulas)
        
    return grammar

choice_a_expr = random.choice

def generate_by_grammar(grammar: dict,target: str):
    if target not in grammar: return target
    expr = choice_a_expr(grammar[target])
#    print(expr)
    return ''.join(generate_by_grammar(grammar, t) for t in expr)

def generate_by_str(grammar_str, split, target):
    grammar = generate_grammar(grammar_str, target, split)
    return generate_by_grammar(grammar, target)



In [4]:
sp={}
for i in range(10):
    a = generate_by_str(eat, split='=>', target='aim')
    sp[i] = a,get_probability(a)
sorted_sp = sorted(list(sp.values()),key=lambda x: x[1]) 
print('{} is more possible with Prb {}'.format(sorted_sp[-1][0],sorted_sp[-1][1]))
for i in range(10):
    print('-'*4 + ' {} with probility {}'.format(sorted_sp[i][0],sorted_sp[i][1]))




今天他吃了青椒凉拌皮蛋，你猜好不好吃 is more possible with Prb 3.179767276616081e-53
---- 昨天我尝了黄豆拌饭酱凉拌茄子，你猜好不好吃 with probility 6.093075079378979e-73
---- 今天我尝了黄豆拌饭酱炒土豆丝，好吃 with probility 4.016029346991201e-67
---- 今天方方尝了木耳凉拌茄子，你猜好不好吃 with probility 1.0809165885202968e-64
---- 昨天红红吃了西红柿炒肉丝，你猜好不好吃 with probility 7.071074409935043e-61
---- 昨天小明尝了青椒凉拌肉丝，不好吃 with probility 8.905586843658199e-60
---- 今天他吃了黄豆拌饭酱炒鸡蛋，你尝尝 with probility 1.3425094745677632e-57
---- 今天红红吃了木耳凉拌茄子，你尝尝 with probility 5.29618422536699e-57
---- 今天龟龟吃了腐竹炒茄子，你尝尝 with probility 5.29618422536699e-57
---- 今天他尝了木耳炒茄子，不好吃 with probility 1.189254486418448e-56
---- 今天他吃了青椒凉拌皮蛋，你猜好不好吃 with probility 3.179767276616081e-53
